In [2]:
# Load necessary library and module
import warnings
warnings.filterwarnings('ignore')

import sys
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import re
import string
import unicodedata
import nltk
import sklearn

nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

%matplotlib inline
# %load_ext google.colab.data_table

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive/', force_remount=True)

In [ ]:
# %cd /content/gdrive/MyDrive/Colab Notebooks/Perintis-GoogleAppsReview/dataset
#MyDrive/Colab Notebooks/Perintis-GoogleAppsReview/dataset

In [3]:
mlbb = pd.read_csv('dataset/mlbb_clean_content_after_preprocessing_20062022.csv',encoding='utf-8')
ffm = pd.read_csv('dataset/ffm_clean_content_after_preprocessing_20062022.csv')#, encoding='ISO-8859-1')
hdi = pd.read_csv('dataset/hdi_clean_content_after_preprocessing_20062022.csv')#, encoding='ISO-8859-1')

In [4]:
mlbb = mlbb[['content','clean_content','score','sentiment','polarity']]
ffm = ffm[['content', 'clean_content','score','sentiment','polarity']]
hdi = hdi[['content', 'clean_content','score','sentiment','polarity']]
#blibli = blibli[['content', 'score']]
# blibli.head()
#mlbb.head()
#ffm.head()

In [8]:
cat_category = [feature for feature in mlbb.columns if mlbb[feature].dtypes == "O"]
mlbb[cat_category].isnull().sum()

content           0
clean_content    35
polarity          0
dtype: int64

In [6]:
cat_category = [feature for feature in ffm.columns if ffm[feature].dtypes == "O"]
ffm[cat_category].isnull().sum()

content           0
clean_content    66
polarity          0
dtype: int64

In [7]:
cat_category = [feature for feature in hdi.columns if hdi[feature].dtypes == "O"]
hdi[cat_category].isnull().sum()

content            0
clean_content    101
polarity           0
dtype: int64

In [10]:
mlbb.isnull().sum()#.values()

content           0
clean_content    35
score             0
sentiment         0
polarity          0
dtype: int64

# Remove NULL Values

In [13]:
mlbb= mlbb[mlbb['clean_content'].notna()]
mlbb['clean_content'].dropna(inplace=True)
# ffm= ffm[ffm['clean_content'].notna(inplace=True)]
# hdi= hdi[hdi['clean_content'].notna(inplace=True)]

In [15]:
# mlbb.shape
mlbb.isnull().sum()#.values()

content          0
clean_content    0
score            0
sentiment        0
polarity         0
dtype: int64

In [ ]:
# ffm.shape

In [ ]:
# hdi.shape

In [16]:
mlbb.info()
mlbb.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1165 entries, 0 to 1199
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   content        1165 non-null   object
 1   clean_content  1165 non-null   object
 2   score          1165 non-null   int64 
 3   sentiment      1165 non-null   int64 
 4   polarity       1165 non-null   object
dtypes: int64(2), object(3)
memory usage: 54.6+ KB


,content,clean_content,score,sentiment,polarity
0,"I'd rate higher but there's a problem, everyti...",rate higher problem everytime hold heros abili...,1,0,negative
1,"just recently, it can't even start a game. It ...",recently even start game always crashed hero s...,1,0,negative
2,"The game is fun, interesting, has a lot of her...",game interesting heroes recently game bugs man...,1,0,negative
3,Its been more than 4 years im playing this gam...,years playing game never went previously whene...,1,0,negative
4,You know what? I don't care anymore. The devel...,know care anymore developer never played aroun...,1,0,negative


In [17]:
mlbb.score.value_counts()


3    387
1    199
2    195
5    195
4    189
Name: score, dtype: int64

In [ ]:
# ffm.score.value_counts()

In [ ]:
# hdi.score.value_counts()

In [18]:
mlbb.sentiment.value_counts()

0    394
1    387
2    384
Name: sentiment, dtype: int64

In [ ]:
# ffm.sentiment.value_counts()

In [ ]:
# hdi.sentiment.value_counts()

In [19]:
class_names = ['negative', 'positive']

# Word Lenght

In [20]:
from tqdm._tqdm_notebook import tqdm_notebook

In [21]:
tqdm_notebook.pandas()
def count_words(x):
    words = word_tokenize(x)
    n=len(words)
    return n

In [22]:
mlbb['word_length'] = mlbb['clean_content'].progress_apply(str).progress_apply(lambda x:count_words(x))

  0%|          | 0/1165 [00:00<?, ?it/s]

  0%|          | 0/1165 [00:00<?, ?it/s]

In [ ]:
# ffm['word_length'] = ffm['clean_content'].progress_apply(str).progress_apply(lambda x:count_words(x))

In [ ]:
# hdi['word_length'] = hdi['clean_content'].progress_apply(str).progress_apply(lambda x:count_words(x))

In [23]:
mlbb['word_length'].value_counts().sort_index()

1     99
2     84
3     57
4     38
5     37
6     21
7     24
8     18
9     29
10    14
11    12
12    12
13    14
14    18
15    17
16    13
17    23
18    23
19    32
20    20
21    22
22    23
23    24
24    36
25    18
26    23
27    20
28    27
29    24
30    20
31    30
32    32
33    18
34    30
35    18
36    22
37    26
38    12
39    18
40    21
41    20
42    25
43    13
44    16
45     4
46     4
47     2
48     1
49     1
51     2
52     2
53     1
55     1
57     1
58     2
63     1
Name: word_length, dtype: int64

In [ ]:
# ffm['word_length'].value_counts().sort_index()

In [ ]:
# hdi['word_length'].value_counts().sort_index()

In [24]:
#sama dengan dropna
mlbb = mlbb.drop(mlbb[mlbb['word_length']==0].index,axis=0)
mlbb = mlbb.reset_index(drop=True)
# ffm = ffm.drop(ffm[ffm['word_length']==0].index,axis=0)
# ffm = ffm.reset_index(drop=True)
# hdi = hdi.drop(hdi[hdi['word_length']==0].index,axis=0)
# hdi = hdi.reset_index(drop=True)

In [25]:
mlbb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1165 entries, 0 to 1164
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   content        1165 non-null   object
 1   clean_content  1165 non-null   object
 2   score          1165 non-null   int64 
 3   sentiment      1165 non-null   int64 
 4   polarity       1165 non-null   object
 5   word_length    1165 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 54.7+ KB


# Word Tokenize

In [ ]:
mlbb_word_dict = {}
for i in range(0,len(mlbb['clean_content'])):
    sentence = mlbb['clean_content'][i]
    word_token = word_tokenize(sentence)
    for j in word_token:
        if j not in mlbb_word_dict:
            mlbb_word_dict[j] = 1
        else:
            mlbb_word_dict[j] += 1

In [ ]:
ffm_word_dict = {}
for i in range(0,len(ffm['clean_content'])):
    sentence = ffm['clean_content'][i]
    word_token = word_tokenize(sentence)
    for j in word_token:
        if j not in ffm_word_dict:
            ffm_word_dict[j] = 1
        else:
            ffm_word_dict[j] += 1

In [ ]:
hdi_word_dict = {}
for i in range(0,len(hdi['clean_content'])):
    sentence = hdi['clean_content'][i]
    word_token = word_tokenize(sentence)
    for j in word_token:
        if j not in hdi_word_dict:
            hdi_word_dict[j] = 1
        else:
            hdi_word_dict[j] += 1

In [ ]:
len(mlbb_word_dict)
print(mlbb_word_dict)
mlbb_wd=pd.Series(mlbb_word_dict)
mlbb_worddict_df = pd.DataFrame({'word':mlbb_wd.index, 'count':mlbb_wd.values})
# mlbb_worddict_df.head()
mlbb_worddict_df.to_csv( "dataset/mlbb_worddict_clean_content_20062022.csv", index=False, encoding='utf-8-sig')

In [ ]:
len(ffm_word_dict)
print(ffm_word_dict)
ffm_wd=pd.Series(ffm_word_dict)
ffm_worddict_df = pd.DataFrame({'word':ffm_wd.index, 'count':ffm_wd.values})
# ffm_worddict_df.head()
ffm_worddict_df.to_csv( "dataset/ffm_worddict_clean_content_20062022.csv", index=False, encoding='utf-8-sig')

In [ ]:
len(lazada_word_dict)
print(lazada_word_dict)
lazada_wd=pd.Series(lazada_word_dict)
lazada_worddict_df = pd.DataFrame({'word':lazada_wd.index, 'count':lazada_wd.values})
# lazada_worddict_df.head()
lazada_worddict_df.to_csv( "Perintis-GoogleAppsReview/lazada_worddict_clean_content_13012022.csv", index=False, encoding='utf-8-sig')

In [ ]:
len({k:v for (k,v) in shopee_word_dict.items() if v < 4})

In [ ]:
len({k:v for (k,v) in toped_word_dict.items() if v < 4})

In [ ]:
len({k:v for (k,v) in lazada_word_dict.items() if v < 4})

# Lexicon Negasi

In [ ]:
negasi = ['bukan','tidak','ga','gk','nggak','kagak','ndak','tdk']
lexicon = pd.read_csv('Perintis-GoogleAppsReview/modified_full_lexicon.csv')
lexicon = lexicon.drop(lexicon[(lexicon['word'] == 'bukan')
                               |(lexicon['word'] == 'tidak')
                               |(lexicon['word'] == 'tdk')
                               |(lexicon['word'] == 'nggak')
                               |(lexicon['word'] == 'kagak')
                               |(lexicon['word'] == 'ndak')
                               |(lexicon['word'] == 'ga')
                               |(lexicon['word'] == 'gk') ].index,axis=0)
shopee = shopee.reset_index(drop=True)
toped = toped.reset_index(drop=True)
lazada = lazada.reset_index(drop=True)

In [ ]:
len(lexicon)
lexicon.head(200)

In [ ]:
lexicon_word = lexicon['word'].to_list()
lexicon_num_words = lexicon['number_of_words']

In [ ]:
len(lexicon_word)

In [ ]:
print(lexicon_word)

In [ ]:
ns_words_shopee = []
factory = StemmerFactory()
stemmer = factory.create_stemmer()
for word in shopee_word_dict.keys():
    if word not in lexicon_word:
        kata_dasar = stemmer.stem(word)
        if kata_dasar not in lexicon_word:
            ns_words_shopee.append(word)
            
len(ns_words_shopee)


In [ ]:
# print(ns_words_shopee)
ns_shopee=pd.DataFrame(ns_words_shopee,columns=['lexicon'])
ns_shopee.to_csv( "Perintis-GoogleAppsReview/dataset/lexicon_shopee_clean_content_oke_13012022.csv", index=False, encoding='utf-8-sig')

In [ ]:
ns_words_toped = []
factory = StemmerFactory()
stemmer = factory.create_stemmer()
for word in toped_word_dict.keys():
    if word not in lexicon_word:
        kata_dasar = stemmer.stem(word)
        if kata_dasar not in lexicon_word:
            ns_words_toped.append(word)
            
len(ns_words_toped)

In [ ]:
# print(ns_words_shopee)
ns_toped=pd.DataFrame(ns_words_toped,columns=['lexicon'])
ns_toped.to_csv( "Perintis-GoogleAppsReview/dataset/lexicon_toped_clean_content_oke_13012022.csv", index=False, encoding='utf-8-sig')

In [ ]:
ns_words_lazada = []
factory = StemmerFactory()
stemmer = factory.create_stemmer()
for word in lazada_word_dict.keys():
    if word not in lexicon_word:
        kata_dasar = stemmer.stem(word)
        if kata_dasar not in lexicon_word:
            ns_words_lazada.append(word)
            
len(ns_words_lazada)

In [ ]:
# print(ns_words_shopee)
ns_lazada=pd.DataFrame(ns_words_lazada,columns=['lexicon'])
ns_lazada.to_csv( "Perintis-GoogleAppsReview/dataset/lexicon_lazada_clean_content_oke_13012022.csv", index=False, encoding='utf-8-sig')

In [ ]:
nswords_data.info()
nswords_data.head()

In [ ]:
len({k:v for (k,v) in shopee_word_dict.items() if ((k in ns_words_shopee)&(v>3)) })

In [ ]:
shopee_ns_words_list = {k:v for (k,v) in shopee_word_dict.items() if ((k in ns_words_shopee)&(v>3))}

In [ ]:
sort_orders = sorted(shopee_ns_words_list.items(), key=lambda x: x[1], reverse=True)
sort_orders=sort_orders[0:50]
for i in sort_orders:
    print(i[0], i[1])

# Wordcloud

In [ ]:
from wordcloud import WordCloud

In [ ]:
shopee_word_to_plot = shopee['clean_content'].copy()
shopee_word_to_plot.head()

In [ ]:

shopee_word_to_plot_df=pd.DataFrame(shopee_word_to_plot)
# print(shopee_word_to_plot)
shopee_word_to_plot_df.head()
shopee_wordcloud = WordCloud(width = 1024, height = 1024, background_color = 'white', max_words = 1200
                      , min_font_size = 20).generate(str(shopee_word_to_plot))
#plot the word cloud
fig = plt.figure(figsize = (8,8), facecolor = None)
plt.imshow(shopee_wordcloud)
plt.axis('off')
plt.show()

In [ ]:
toped_word_to_plot = toped['clean_content'].copy()
toped_word_to_plot_df=pd.DataFrame(toped_word_to_plot)
# print(toped_word_to_plot)
toped_word_to_plot_df.head()
toped_wordcloud = WordCloud(width = 1024, height = 1024, background_color = 'white', max_words = 1200
                      , min_font_size = 20).generate(str(toped_word_to_plot))
#plot the word cloud
fig = plt.figure(figsize = (8,8), facecolor = None)
plt.imshow(toped_wordcloud)
plt.axis('off')
plt.show()

In [ ]:
lazada_word_to_plot = lazada['clean_content'].copy()
lazada_word_to_plot_df=pd.DataFrame(lazada_word_to_plot)
# print(lazada_word_to_plot)
lazada_word_to_plot_df.head()
lazada_wordcloud = WordCloud(width = 1024, height = 1024, background_color = 'white', max_words = 1200
                      , min_font_size = 20).generate(str(lazada_word_to_plot))
#plot the word cloud
fig = plt.figure(figsize = (8,8), facecolor = None)
plt.imshow(lazada_wordcloud)
plt.axis('off')
plt.show()

# Lexicon

In [ ]:
toped.head(300)

In [ ]:
lexicon['number_of_words'].value_counts()

In [ ]:
cek_shopee = pd.DataFrame([])
cek_shopee['content'] = shopee['content'].copy()
cek_shopee['clean_content'] = shopee['clean_content'].copy()
cek_shopee['score'] = shopee['score'].copy()
cek_shopee['sentiment']  = shopee['sentiment'].copy() #cek sentimens sebelum diproses lexicon to sentimen
cek_shopee.sample(200)

In [ ]:
sns.set(style="white", palette="muted", color_codes=True)
sns.kdeplot(shopee['sentiment'],color='r',shade=True)
plt.title('Sentiment Distribution')
plt.xlabel('sentiment')

In [ ]:
sns.set(style="whitegrid") 
sns.boxplot(x=shopee['sentiment'])

In [ ]:
df=shopee
sencol =[]
senrow =np.array([])
nsen = 0
factory = StemmerFactory()
stemmer = factory.create_stemmer()
sentiment_list = []
# function to write the word's sentiment if it is founded
def found_word(ind,words,word,sen,sencol,sentiment,add):
    # if it is already included in the bag of words matrix, then just increase the value
    if word in sencol:
        sen[sencol.index(word)] += 1
    else:
    #if not, than add new word
        sencol.append(word)
        sen.append(1)
        add += 1
    #if there is a negation word before it, the sentiment would be the negation of it's sentiment
    if (words[ind-1] in negasi):
        sentiment += -lexicon['weight'][lexicon_word.index(word)]
    else:
        sentiment += lexicon['weight'][lexicon_word.index(word)]
    
    return sen,sencol,sentiment,add
            
# checking every words, if they are appear in the lexicon, and then calculate their sentiment if they do
for i in range(len(df)):
    nsen = senrow.shape[0]
    words = word_tokenize(df['clean_content'][i])
    sentiment = 0 
    add = 0
    prev = [0 for ii in range(len(words))]
    n_words = len(words)
    if len(sencol)>0:
        sen =[0 for j in range(len(sencol))]
    else:
        sen =[]
    
    for word in words:
        ind = words.index(word)
        # check whether they are included in the lexicon
        if word in lexicon_word :
            sen,sencol,sentiment,add= found_word(ind,words,word,sen,sencol,sentiment,add)
        else:
        # if not, then check the root word
            kata_dasar = stemmer.stem(word)
            if kata_dasar in lexicon_word:
                sen,sencol,sentiment,add= found_word(ind,words,kata_dasar,sen,sencol,sentiment,add)
        # if still negative, try to match the combination of words with the adjacent words
            elif(n_words>1):
                if ind-1>-1:
                    back_1    = words[ind-1]+' '+word
                    if (back_1 in lexicon_word):
                        sen,sencol,sentiment,add= found_word(ind,words,back_1,sen,sencol,sentiment,add)
                    elif(ind-2>-1):
                        back_2    = words[ind-2]+' '+back_1
                        if back_2 in lexicon_word:
                            sen,sencol,sentiment,add= found_word(ind,words,back_2,sen,sencol,sentiment,add)
    # if there is new word founded, then expand the matrix
    if add>0:  
        if i>0:
            if (nsen==0):
                senrow = np.zeros([i,add],dtype=int)
            elif(i!=nsen):
                padding_h = np.zeros([nsen,add],dtype=int)
                senrow = np.hstack((senrow,padding_h))
                padding_v = np.zeros([(i-nsen),senrow.shape[1]],dtype=int)
                senrow = np.vstack((senrow,padding_v))
            else:
                padding =np.zeros([nsen,add],dtype=int)
                senrow = np.hstack((senrow,padding))
            senrow = np.vstack((senrow,sen))
        if i==0:
            senrow = np.array(sen).reshape(1,len(sen))
    # if there isn't then just update the old matrix
    elif(nsen>0):
        senrow = np.vstack((senrow,sen))
        
    sentiment_list.append(sentiment)

# Define Feature Matrix and Target Matrix

Split Training And Testing Dataset

In [ ]:

X = mlbb['clean_content']     # Define feature matrix
y = mlbb['sentiment']         # Define target feature matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=0, stratify=y)

print('[INFO] Sebaran kelas pada training set: negatif={}, netral={}, dan positif={}'.format(sum(y_train==0), sum(y_train==1), sum(y_train==2)))
print('[INFO] Sebaran kelas pada testing set negatif={}, netral={}, dan positif={}'.format(sum(y_test==0), sum(y_test==1), sum(y_test==2)))

X_train.shape, X_test.shape, y_train.shape, y_test.shape



Training dan Testing Tokopedia

In [ ]:
X_train.head()
X_train_data=pd.DataFrame(X_train)
X_test_data=pd.DataFrame(X_test)

In [ ]:
#X_test.head()
X_train_data.head()
#shopee.clean_content.head()

In [ ]:
#ngram_range=(1,2) #bigram
ngram_range=(1,3) #trigram
#ngram_range=(1,2) #bigram
min_df=5

tfidf=TfidfVectorizer(ngram_range=ngram_range,min_df=min_df)
# tfidf=TfidfVectorizer(max_features=1000)

# Term Weighting TFIDF X Training

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer()

In [ ]:
mlbb_vector_x_train=tfidf.fit_transform(X_train).toarray()
# mlbb_vector_x_train_cv=cv.fit_transform(X_train).toarray()
mlbb_x_train_df=pd.DataFrame(mlbb_vector_x_train,columns=tfidf.get_feature_names())
# mlbb_x_train_cv_df=pd.DataFrame(mlbb_vector_x_train_cv)
# mlbb_x_train_cv_df.head()


In [ ]:
mlbb_x_train_df.info()
# toped_x_train_df.info()
# lazada_x_train_df.info()

# Term Weighting TFIDF X Testing

Tokopedia

In [ ]:
mlbb_vector_x_test=tfidf.transform(X_test).toarray()
mlbb_x_test_df=pd.DataFrame(mlbb_vector_x_test,columns=tfidf.get_feature_names())
mlbb_x_test_df.head()
#toped_df.head()

In [ ]:
from sklearn.svm import SVC

# Linear Model

In [ ]:
svc_model = SVC(C= .1, kernel='linear', gamma= 1)
clf.fit(mlbb_vector_x_train, y_train)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = svc_model.predict(mlbb_vector_x_train)
# train_acc = accuracy_score(y_train, svc_model.predict(mlbb_vector_x_train))
print(classification_report(y_train, train_acc))
print()
print("Confusion Matrix: \n", confusion_matrix(y_train, pred))
print("Accuracy: \n", accuracy_score(y_train, pred))

# Training

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier

In [ ]:
C=0.7
# Create and train a random forest classifier
# forest = RandomForestClassifier()
models = [
          # ('k-NN', KNeighborsClassifier()),
          ('SVC linear kernel', SVC(kernel='linear',C=C,max_iter=10000)),
          ('SVC RBF kernel', SVC(kernel='rbf',gamma=1,C=C)), 
          ('SVC Polynomial (degree 3)', SVC(kernel='poly',degree=3,C=C)),
          ('SVC Sigmoid', SVC(kernel='sigmoid',C=C)),
          # ('Naive Bayes', MultinomialNB()),
          # ('Decision Tree', DecisionTreeClassifier()),
          # ('Random Forest',RandomForestClassifier(n_estimators=50)),
          # ('Regresi Linier',LogisticRegression()),
        ]


In [ ]:
for name, clf in models:
  clf.fit(mlbb_vector_x_train, y_train)
  train_acc = accuracy_score(y_train, clf.predict(mlbb_vector_x_train))
  print('[INFO] Menggunakan {}, akurasi pada training set: {}'.format(name, train_acc))

# Cross Validation

In [ ]:
# cross-validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

for name, model in models:
  clf.fit(mlbb_vector_x_train, y_train)
  cv = KFold(n_splits=5, random_state=27, shuffle=True)
  cross_val_acc = cross_val_score(clf, mlbb_vector_x_train, y_train, cv=cv, scoring='accuracy')
  print('[INFO] Menggunakan {}, Cross-validation akurasi: {:5f} (+/- {:5f})'.format(name, cross_val_acc.mean(), cross_val_acc.std() * 2))

Training Confusion Matrix

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer()

# X_train = cv.fit_transform(shopee_vector_x_train.clean_content)

# X_test = cv.transform(shopee_vector_x_test.clean_content)

In [ ]:
from sklearn.metrics import plot_confusion_matrix

for name, model in models:
  plot_confusion_matrix(model, mlbb_vector_x_train, y_train) 
  plt.title('Confusion Matrix: {}'.format(name))
  plt.show()

# Testing

In [ ]:
for name, clf in models:
  clf_predict = clf.predict(mlbb_vector_x_test)
  test_acc = accuracy_score(y_test, clf.predict(mlbb_vector_x_test))
  print('[INFO] Menggunakan {}, akurasi pada testing set: {}'.format(name, test_acc))

Laporan Testing

In [ ]:
print("[INFO] Laporan klasifikasi\n")
for name, clf in models:
  clf_predict = clf.predict(mlbb_vector_x_test)
  print('[INFO] Menggunakan {}'.format(name))
  print(classification_report(y_test, clf_predict, target_names=['negative', 'neutral', 'positive']))

Testing Confusion Matrix

In [ ]:
for name, model in models:
  
  plot_confusion_matrix(model, toped_vector_x_test, y_test) 
  plt.title('Confusion Matrix Test: {}'.format(name))
  
  plt.show()